<a href="https://www.kaggle.com/code/lucascarpantonio/titanic-machine-learning-from-disaster?scriptVersionId=286191111" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a href="https://www.kaggle.com/code/lucascarpantonio/titanic-machine-learning-from-disaster?scriptVersionId=285818002" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


### 1. Overview

In [2]:
# loading training data (use local Kaggle input path)
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
train_data.info()
train_data.head(4)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


In [3]:
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
test_data.info()
test_data.head(4)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S


In [4]:
train_X = train_data.drop(columns='Survived', errors='ignore')

train_y = train_data['Survived']


train_vs_test = pd.DataFrame({
	'train': pd.Series(train_X.columns.tolist()),
	'test': pd.Series(test_data.columns.tolist())
})

train_vs_test


,train,test
0,PassengerId,PassengerId
1,Pclass,Pclass
2,Name,Name
3,Sex,Sex
4,Age,Age
5,SibSp,SibSp
6,Parch,Parch
7,Ticket,Ticket
8,Fare,Fare
9,Cabin,Cabin


## 3. Build Model

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

num = ['PassengerId','Age', 'Pclass','Fare', 'Parch'] #remove 'SibSp' to avoid redundancy
cat = ['Sex','Ticket','Embarked']

feature_cols = num + cat


X = train_data[feature_cols]
y = train_data['Survived']

In [6]:

# 2) Trasformazioni
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median"))
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num),
        ("cat", categorical_transformer, cat),
    ]
)

In [7]:
# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(train_X, train_y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)


In [8]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=90,
    random_state=42
)

In [9]:
# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train_full, y_train)

# Preprocessing of validation data, get predictions
preds_valid = my_pipeline.predict(X_valid_full)


In [10]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(
    my_pipeline, X_train_full, y_train,
    cv=5,
    scoring="accuracy"
)

scores.mean(), scores.std()

(0.8145966709346991, 0.01811505004198514)

In [11]:
from sklearn.metrics import mean_absolute_error


mae = mean_absolute_error(y_valid, preds_valid)
print("MAE:", mae)

MAE: 0.1452513966480447


### 4. Prediction

In [12]:
my_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", model)
])

my_pipeline.fit(X[num + cat], y)

preds = my_pipeline.predict(test_data[num + cat])

### 5. Conclusions

With the final model being trained on the full `train_data` and used to generate survival predictions for the Kaggle `test_data`, the predicted labels were recombined with the original test dataset.  
This step allows us to **verify whether the survival patterns learned by the model are consistent with the trends observed in the training set**.

Since the true `Survived` values are not available for `test_data`, this comparison cannot measure accuracy.  
Instead, it provides a **qualitative sanity check**: we compare key demographic distributions (such as Age) for predicted survivors and non-survivors against their real counterparts in the training data.

If the model has captured meaningful patterns, the predicted distributions should resemble those found in the original dataset.  
The following boxplot illustrates this comparison. Despite natural differences, the overall survival trends across age groups appear coherent between the real (training) and predicted (test) populations, suggesting that the model has internalized the general dynamics of survival on the Titanic.

In [13]:
output = pd.DataFrame({
    'PassengerId': test_data['PassengerId'],
    'Survived': preds
})
output.to_csv('submission.csv', index=False)
